In [20]:
import pandas as pd
import numpy as np
import re

import plotly.express as px
import datetime
from Project.Database import Db
from Project._05InferKnowledgeOfRules.infer_rules_functions import SE_time_df, json_to_dataframe
from Project._07OptimiseConsumption.optimisation_problem import NZERTF_optimiser, hourly_house_df

In [21]:
NZERTF_after = NZERTF_optimiser(year=2)

In [22]:
NZERTF_before, NZERTF_meta = Db.load_data(hourly=False, meta=True, year=2, with_redundancy=True, consumption=True)

In [23]:
NZERTF_before

,Timestamp,Load_LatentHeatWaterVolume,Load_RefrigeratorTemp,Load_StatusBA1Lights,Load_StatusKitchenLightsA,Load_StatusKitchenLightsB,Load_StatusKitchenLightsC,Load_StatusDRLights,Load_StatusLRLights3,Load_StatusEntryHallLights,...,HVAC_HeatPumpOutdoorUnitPower,HVAC_DehumidifierPower,HVAC_DehumidifierInletAirTemp,HVAC_DehumidifierExitAirTemp,HVAC_DehumidifierAirflow,Load_StatusRefrigerator,Load_StatusMicrowave,Load_StatusClothesWasher,Load_StatusDryerPowerTotal,Load_StatusHeatPumpWaterHeater
0,2015-02-01 00:01:50,0.000000,4.994715,0,0,0,0,0,0,0,...,1108.765,4.471,72.102,74.655,0.0,0,0,0,0,0
1,2015-02-01 00:02:50,0.000000,5.046792,0,0,0,0,0,0,0,...,1097.577,4.508,72.096,74.701,0.0,0,0,0,0,0
2,2015-02-01 00:03:50,0.000000,5.129493,0,0,0,0,0,0,0,...,1086.113,4.510,72.077,74.759,0.0,0,0,0,0,0
3,2015-02-01 00:04:50,0.001981,5.163539,0,0,0,0,0,0,0,...,1079.586,4.529,72.074,74.829,0.0,0,0,0,0,0
4,2015-02-01 00:05:50,0.001981,5.232744,0,0,0,0,0,0,0,...,1048.502,4.438,72.086,74.863,0.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519599,2016-01-31 23:54:24,1.757590,4.512704,0,0,0,0,0,0,0,...,22.563,4.265,67.645,68.550,0.0,0,0,0,0,0
519600,2016-01-31 23:55:24,1.760893,4.514255,0,0,0,0,0,0,0,...,22.475,4.286,67.655,68.551,0.0,0,0,0,0,0
519601,2016-01-31 23:56:24,1.763535,4.548853,0,0,0,0,0,0,0,...,22.388,4.293,67.644,68.549,0.0,0,0,0,0,0
519602,2016-01-31 23:57:24,1.763535,4.557287,0,0,0,0,0,0,0,...,22.169,4.306,67.651,68.552,0.0,0,0,0,0,0


In [24]:
NZERTF_before[['Timestamp','Load_StatusPlugLoadVacuum']].loc[lambda self: (self['Timestamp'].dt.date == pd.to_datetime('2015-05-16').date()) & self['Load_StatusPlugLoadVacuum']==1]

,Timestamp,Load_StatusPlugLoadVacuum
150544,2015-05-16 17:29:53,1
150545,2015-05-16 17:30:53,1
150546,2015-05-16 17:31:53,1
150547,2015-05-16 17:32:53,1
150548,2015-05-16 17:33:53,1
...,...,...
150649,2015-05-16 19:14:53,1
150650,2015-05-16 19:15:53,1
150651,2015-05-16 19:16:53,1
150652,2015-05-16 19:17:53,1


In [25]:
NZERTF_before[['Timestamp','Load_StatusPlugLoadVacuum']].loc[lambda self: self['Load_StatusPlugLoadVacuum']==1]['Timestamp'].dt.date.unique()

array([datetime.date(2015, 2, 7), datetime.date(2015, 2, 14),
       datetime.date(2015, 2, 21), datetime.date(2015, 2, 28),
       datetime.date(2015, 3, 7), datetime.date(2015, 3, 14),
       datetime.date(2015, 3, 21), datetime.date(2015, 3, 28),
       datetime.date(2015, 4, 4), datetime.date(2015, 4, 11),
       datetime.date(2015, 4, 18), datetime.date(2015, 4, 25),
       datetime.date(2015, 5, 2), datetime.date(2015, 5, 9),
       datetime.date(2015, 5, 16), datetime.date(2015, 5, 23),
       datetime.date(2015, 5, 30), datetime.date(2015, 6, 6),
       datetime.date(2015, 6, 13), datetime.date(2015, 6, 20),
       datetime.date(2015, 6, 27), datetime.date(2015, 7, 4),
       datetime.date(2015, 7, 11), datetime.date(2015, 7, 18),
       datetime.date(2015, 7, 25), datetime.date(2015, 8, 1),
       datetime.date(2015, 8, 8), datetime.date(2015, 8, 15),
       datetime.date(2015, 8, 22), datetime.date(2015, 8, 29),
       datetime.date(2015, 9, 5), datetime.date(2015, 9, 12),
  

In [26]:
NZERTF_before['Timestamp'] = NZERTF_before['Timestamp'] + pd.DateOffset(hours=0, minutes=1)

In [27]:
NZERTF_before_hourly = hourly_house_df(NZERTF_before, aggregate_func='max')

In [28]:
production = Db.load_data(hourly=False, meta=False, year=2, with_redundancy=False, consumption=False, production=True)['CO2(Grams)/kWh']

In [29]:
NZERTF_before_hourly_with_prod = NZERTF_before_hourly.merge(production, left_on='Timestamp', right_on=production.index)

In [30]:
NZERTF_after_with_prod = NZERTF_after.merge(production, left_on='Timestamp', right_on=production.index)

In [31]:
NZERTF_before.columns

Index(['Timestamp', 'Load_LatentHeatWaterVolume', 'Load_RefrigeratorTemp',
       'Load_StatusBA1Lights', 'Load_StatusKitchenLightsA',
       'Load_StatusKitchenLightsB', 'Load_StatusKitchenLightsC',
       'Load_StatusDRLights', 'Load_StatusLRLights3',
       'Load_StatusEntryHallLights',
       ...
       'HVAC_DehumidifierInletAirTemp', 'HVAC_DehumidifierExitAirTemp',
       'HVAC_DehumidifierAirflow', 'Load_StatusRefrigerator',
       'Load_StatusMicrowave', 'Load_StatusClothesWasher',
       'Load_StatusDryerPowerTotal', 'Load_StatusHeatPumpWaterHeater', 'Day',
       'Hour'],
      dtype='object', length=387)

In [32]:
NZERTF_after

,Day,Hour,Timestamp,Load_StatusApplianceDishwasher,Load_StatusPlugLoadVacuum,Load_StatusClothesWasher,Load_StatusDryerPowerTotal,Load_StatusPlugLoadIron,Emission,CumulativeEmission
0,0,0,2015-02-01 00:00:00,0,0,0,0,0,0.000000,0.000000
1,0,1,2015-02-01 01:00:00,0,0,0,0,0,0.000000,0.000000
2,0,2,2015-02-01 02:00:00,0,0,0,0,0,0.000000,0.000000
3,0,3,2015-02-01 03:00:00,0,0,0,0,0,0.000000,0.000000
4,0,4,2015-02-01 04:00:00,0,0,0,0,0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
8667,364,19,2016-01-31 19:00:00,0,0,0,0,0,0.000000,538303.572346
8668,364,20,2016-01-31 20:00:00,0,0,1,0,0,48.797821,538352.370166
8669,364,21,2016-01-31 21:00:00,1,0,1,0,0,1069.285258,539421.655424
8670,364,22,2016-01-31 22:00:00,1,0,0,0,0,118.589460,539540.244884


In [33]:
appliance_job_list = []
for df in [NZERTF_before_hourly_with_prod, NZERTF_after_with_prod]:
    #Extract appliances from the meta data
    appliance_condition = [x for x in NZERTF_after.columns.tolist() if "Status" in x]
    appliance_location = pd.DataFrame(NZERTF_meta.loc[appliance_condition, "Measurement_Location"].sort_values(ascending=True))
    #Combine minute data with the extracted appliances into a dataframe
    minute_appliances_status = df[["Timestamp"] + appliance_location.index.tolist()].copy()

    for appliance, appliance_row in appliance_location.iterrows():
        name = " ".join(re.findall('[A-Z][^A-Z]*', appliance.split("Load_Status")[-1]))
        appliance_switch = minute_appliances_status[(minute_appliances_status[appliance] != minute_appliances_status[appliance].shift(1))][["Timestamp", appliance]][1:]
        appliance_switch.reset_index(inplace=True, drop=True)
        for index, row in appliance_switch.iterrows():
            if row[appliance]:
                try:
                    start = row["Timestamp"]
                    end = appliance_switch.loc[index + 1, "Timestamp"]
                    start_index = df.loc[lambda self: self['Timestamp']==start].index.tolist()[0]
                    end_index = df.loc[lambda self: self['Timestamp']==end].index.tolist()[0]
                    emission = round(df[start_index:end_index+1]['CO2(Grams)/kWh'].mean() / 1000, 2)
                    if 'Load_StatusKitchenLightsA' in df:
                        appliance_job_list.append({"appliance": name+' before', "start": row["Timestamp"],
                                                       "end": appliance_switch.loc[index + 1, "Timestamp"],
                                                       "location": appliance_row["Measurement_Location"],
                                                   'Emission (CO2)kg/kWh': emission})
                    else:
                        appliance_job_list.append({"appliance": name+' after',
                                                   "start": start,
                                                   "end": end,
                                                   "location": appliance_row["Measurement_Location"],
                                                   'Emission (CO2)kg/kWh': emission})
                except:
                    continue

In [34]:
new_appliance_job_list = []
for i in appliance_job_list:
    if 'Clothes Washer after' in i['appliance']:
        i['appliance'] = 'Clothes Washer and Dryer after'
    new_appliance_job_list.append(i)
new_appliance_job_list

[{'appliance': 'Plug Load Vacuum before',
  'start': Timestamp('2015-02-07 17:00:00'),
  'end': Timestamp('2015-02-07 20:00:00'),
  'location': 'Bedroom4',
  'Emission (CO2)kg/kWh': 0.72},
 {'appliance': 'Plug Load Vacuum before',
  'start': Timestamp('2015-02-14 17:00:00'),
  'end': Timestamp('2015-02-14 20:00:00'),
  'location': 'Bedroom4',
  'Emission (CO2)kg/kWh': 0.89},
 {'appliance': 'Plug Load Vacuum before',
  'start': Timestamp('2015-02-21 17:00:00'),
  'end': Timestamp('2015-02-21 20:00:00'),
  'location': 'Bedroom4',
  'Emission (CO2)kg/kWh': 0.61},
 {'appliance': 'Plug Load Vacuum before',
  'start': Timestamp('2015-02-28 17:00:00'),
  'end': Timestamp('2015-02-28 20:00:00'),
  'location': 'Bedroom4',
  'Emission (CO2)kg/kWh': 0.74},
 {'appliance': 'Plug Load Vacuum before',
  'start': Timestamp('2015-03-07 17:00:00'),
  'end': Timestamp('2015-03-07 20:00:00'),
  'location': 'Bedroom4',
  'Emission (CO2)kg/kWh': 0.7},
 {'appliance': 'Plug Load Vacuum before',
  'start': Tim

In [35]:
%%time
fig = px.timeline(new_appliance_job_list, x_start="start", x_end="end", y="appliance", color='Emission (CO2)kg/kWh', text='Emission (CO2)kg/kWh')
fig.update_layout(template='plotly')

CPU times: user 74.1 ms, sys: 4.33 ms, total: 78.4 ms
Wall time: 76.5 ms


In [36]:
fig = px.line(production, x=production.index, y="CO2(Grams)/kWh",
        line_shape="spline", render_mode="svg")
fig.show()